In [ ]:
import numpy as np

import matplotlib.pyplot as plt

ガウス分布のシグナル疑似データと指数分布のバックグラウンドデータを生成します。

In [ ]:
x_sig = np.random.normal(200, 30, 5000)
x_bg = np.random.exponential(100, 10000)
x = np.concatenate([x_sig, x_bg], 0)

ヒストグラムとして表示

In [ ]:
n, binsx = np.histogram(x, range=(0, 500), bins=500)
binsx = (binsx[:-1]+binsx[1:])*0.5
plt.plot(binsx, n)

`curve_fit()`で用いるモデル関数を定義します。  
ここではシグナルのみ、バックグラウンドのみ、シグナル＋バックグラウンドの３つのモデル関数を用意します。

In [ ]:
from scipy.optimize import curve_fit


def sig(x, a, m, s):
    return a*np.exp(-0.5*(x-m)**2/s**2)


def bg(x, b, beta):
    return b*np.exp(-x/beta)


def fun(x, a, m, s, b, beta):
    return a*np.exp(-0.5*(x-m)**2/s**2)+b*np.exp(-x/beta)

はじめにシグナル領域のみのフィッティングを行います。

In [ ]:
x1 = binsx[(100 < binsx) & (binsx < 300)]
y1 = n[(100 < binsx) & (binsx < 300)]
par1, cov = curve_fit(sig, x1, y1, p0=(400, 100, 100))

fig = plt.figure(figsize=(6, 4))
y = sig(binsx, *par1)
ax = fig.add_subplot(111)
_ = ax.hist(x, bins=500, range=(0, 500), alpha=0.5, label='data')
_ = ax.plot(binsx, y, '-', label='fit')
plt.legend(loc='upper right')

次にバックグラウンド領域のみのフィッティングを行います。

In [ ]:
x2 = binsx[binsx < 100]
y2 = n[binsx < 100]
par2, cov = curve_fit(bg, x2, y2, p0=(400, 100))

fig = plt.figure(figsize=(6, 4))
y = bg(binsx, *par2)
ax = fig.add_subplot(111)
_ = ax.hist(x, bins=500, range=(0, 500), alpha=0.5, label='data')
_ = ax.plot(binsx, y, '-', label='fit')
plt.legend(loc='upper right')

最後に各領域のフィッティングで得られた最適値を初期パラメータとして全領域をシグナル＋バッググラウンド関数でフィッティングします。

In [ ]:
print(par1, par2)

In [ ]:
par_ini = np.concatenate([par1, par2], 0)
par, cov = curve_fit(fun, binsx, n, p0=par_ini)
print(par)

In [ ]:
fig = plt.figure(figsize=(6, 4))
y = fun(binsx, *par)
ax = fig.add_subplot(111)
_ = ax.hist(x, bins=500, range=(0, 500), alpha=0.5, label='data')
_ = ax.plot(binsx, y, '-', label='fit')
plt.legend(loc='upper right')